In [9]:
import datetime
import pandas as pd
import requests
import time

In [34]:
#input required ERC-20 tokens
token_addresses = ["0x9f8F72aA9304c8B593d555F12eF6589cC3A579A2","0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599"]

In [35]:

vs_currencies = ['usd'] # Replace with the currencies you want
days_per_request = 90 # Maximum number of days per API request
start_date = datetime.datetime.strptime('2020-01-01', '%Y-%m-%d')
end_date = datetime.datetime.strptime('2020-04-30', '%Y-%m-%d')

# Format the start and end dates for the API request
start_date_str = start_date.strftime("%d-%m-%Y")
end_date_str = end_date.strftime("%d-%m-%Y")

# Make the API requests
price_data = {}
api_calls = 0
for token_address in token_addresses:
    try:
        price_data[token_address] = {}
        current_date = start_date
        while current_date < end_date:
            days = min((end_date - current_date).days, days_per_request)
            url = f"https://api.coingecko.com/api/v3/coins/ethereum/contract/{token_address}/market_chart/range?vs_currency={','.join(vs_currencies)}&from={current_date.strftime('%s')}&to={(current_date + datetime.timedelta(days=days)).strftime('%s')}"
            response = requests.get(url)
            response_json = response.json()
            api_calls += 1
            #print(f"API call {api_calls} made")
            for price in response_json['prices']:
                timestamp = price[0]
                price_value = price[1]
                date_str = datetime.datetime.fromtimestamp(timestamp/1000).strftime("%Y-%m-%d")
                price_data[token_address][date_str] = price_value
            current_date += datetime.timedelta(days=days)
            time.sleep(4) #set a delay betweeen API calls to not exceed free request limits.
    except KeyError:
        print(f"No price data found for token address {token_address}. Skipping...")
        continue


In [38]:
#now store the result in a csv
data_rows = []
for token_address in price_data:
    for date, price in price_data[token_address].items():
        data_rows.append([token_address, date, price])

df = pd.DataFrame(data_rows, columns=['token-address', 'date', 'price_usd'])

# Write DataFrame to a CSV file
df.to_csv('crypto-price-data.csv', index=False)